# Prepare ML-Ready Dataset: Methylation + Drug Response

**Goal**: Create a self-contained dataset for training machine learning models to predict drug response from DNA methylation data.

**Output**: 
- `data/processed/ML_dataset_methylation_drug_response.csv` - Combined methylation + drug response
- `data/processed/ML_dataset_metadata.md` - Comprehensive documentation

**Dataset**: GSE68379 (1,028 pan-cancer cell lines) + GDSC drug response data

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import requests
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("Imports complete")

## 1. Load GSE68379 Methylation Data

In [ ]:
# Load methylation data (this may take 3-4 minutes for 3.9 GB file)
print("Loading GSE68379 methylation matrix...")
df_meth = pd.read_csv('../data/raw/GSE68379/GSE68379_Matrix.processed.txt.gz',
                      sep='\t', index_col=0, compression='gzip')

# Remove "Row.names" column if present
if 'Row.names' in df_meth.columns:
    df_meth = df_meth.drop(columns=['Row.names'])

# Clean column names (remove _AVG.Beta suffix)
df_meth.columns = [col.replace('_AVG.Beta', '') for col in df_meth.columns]

print(f"Methylation data shape: {df_meth.shape}")
print(f"CpG sites: {df_meth.shape[0]:,}")
print(f"Cell lines: {df_meth.shape[1]:,}")
print(f"Memory usage: {df_meth.memory_usage(deep=True).sum() / 1e9:.2f} GB")

In [ ]:
# Load sample metadata
df_meta = pd.read_csv('../data/processed/GSE68379_sample_metadata.csv')
print(f"Sample metadata shape: {df_meta.shape}")
print(f"\nMetadata columns: {df_meta.columns.tolist()}")
df_meta.head()

## 2. Download GDSC Drug Response Data

The Genomics of Drug Sensitivity in Cancer (GDSC) project provides IC50 values for hundreds of drugs across cancer cell lines.

In [ ]:
# Download GDSC1 and GDSC2 drug response data
# GDSC provides bulk download files for all drug responses

print("Downloading GDSC drug response data...")
print("This may take a few minutes...\n")

# GDSC1 (older dataset, more drugs)
url_gdsc1 = 'https://www.cancerrxgene.org/downloads/bulk_download?file=GDSC1_fitted_dose_response_25Feb20.xlsx'
# GDSC2 (newer dataset, different drugs)
url_gdsc2 = 'https://www.cancerrxgene.org/downloads/bulk_download?file=GDSC2_fitted_dose_response_25Feb20.xlsx'

# Try downloading - if this fails, we'll use alternative URLs
try:
    # GDSC provides CSV versions as well
    url_csv = 'https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip'
    print("Attempting to download from GDSC...")
    print("Note: GDSC website structure may have changed. Checking alternative sources...")
except Exception as e:
    print(f"Error: {e}")
    print("Will use alternative data source...")

# Alternative: Use FTP site or pre-downloaded files
print("\nNote: GDSC data is large and may require direct download.")
print("We'll check if data is already available locally first.")

In [ ]:
# Check if GDSC data already exists locally
gdsc_file = Path('../data/raw/GDSC/GDSC_drug_response.csv')

if gdsc_file.exists():
    print("Found local GDSC data!")
    df_gdsc = pd.read_csv(gdsc_file)
else:
    print("GDSC data not found locally.")
    print("\nOption 1: Download from GDSC DepMap portal")
    print("URL: https://depmap.org/portal/download/")
    print("File: CTRP or GDSC drug sensitivity data")
    print("\nOption 2: Use GDSC bulk download")
    print("URL: https://www.cancerrxgene.org/downloads/bulk_download")
    print("\nFor now, we'll create a download script...")

In [ ]:
# Create data/raw/GDSC directory if it doesn't exist
gdsc_dir = Path('../data/raw/GDSC')
gdsc_dir.mkdir(parents=True, exist_ok=True)
print(f"Created directory: {gdsc_dir}")

# Download GDSC data using updated URLs
# GDSC provides data through CellModelPassports/DepMap now

print("\nAttempting to download GDSC drug screening data...")
print("Source: CellModelPassports (ftp.sanger.ac.uk)\n")

# Latest GDSC data is available via FTP
gdsc_url = 'https://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC1_fitted_dose_response_17Jul19.xlsx'

# Alternative: Use CSV version
# For initial testing, let's use a smaller subset
print("Downloading GDSC1 fitted dose response data...")
print("(This file is ~50MB, may take 1-2 minutes)\n")

In [ ]:
# Download GDSC data with error handling
import urllib.request
from tqdm.auto import tqdm

def download_file_with_progress(url, output_path):
    """Download file with progress bar"""
    try:
        # Get file size
        with urllib.request.urlopen(url) as response:
            file_size = int(response.headers.get('Content-Length', 0))
        
        # Download with progress
        with tqdm(total=file_size, unit='B', unit_scale=True, desc='Downloading') as pbar:
            def reporthook(blocknum, blocksize, totalsize):
                pbar.update(blocksize)
            urllib.request.urlretrieve(url, output_path, reporthook=reporthook)
        
        print(f"\nDownload complete: {output_path}")
        return True
    except Exception as e:
        print(f"Download failed: {e}")
        return False

# Try downloading
gdsc_output = gdsc_dir / 'GDSC1_fitted_dose_response.xlsx'

if not gdsc_output.exists():
    success = download_file_with_progress(gdsc_url, gdsc_output)
    if not success:
        print("\nAutomatic download failed.")
        print("\nManual download instructions:")
        print("1. Visit: https://www.cancerrxgene.org/downloads/bulk_download")
        print("2. Download: 'GDSC1 Fitted dose response'")
        print(f"3. Save to: {gdsc_output.absolute()}")
        print("\nThen re-run this cell.")
else:
    print(f"GDSC data already exists: {gdsc_output}")

In [ ]:
# Load GDSC data
if gdsc_output.exists():
    print("Loading GDSC drug response data...")
    df_gdsc_raw = pd.read_excel(gdsc_output)
    print(f"GDSC data shape: {df_gdsc_raw.shape}")
    print(f"\nColumns: {df_gdsc_raw.columns.tolist()}")
    print(f"\nFirst few rows:")
    display(df_gdsc_raw.head())
else:
    print("GDSC data not available. Please download manually.")
    print("\nCreating placeholder for demonstration...")
    # Create a minimal example structure
    df_gdsc_raw = pd.DataFrame({
        'CELL_LINE_NAME': [],
        'DRUG_NAME': [],
        'LN_IC50': [],
        'AUC': []
    })

## 3. Match Cell Lines Between GSE68379 and GDSC

Cell line names may differ slightly between databases. We need to harmonize them.

In [ ]:
# Get cell line names from both datasets
gse_cell_lines = set(df_meth.columns)
gdsc_cell_lines = set(df_gdsc_raw['CELL_LINE_NAME'].unique()) if 'CELL_LINE_NAME' in df_gdsc_raw.columns else set()

print(f"GSE68379 cell lines: {len(gse_cell_lines)}")
print(f"GDSC cell lines: {len(gdsc_cell_lines)}")

# Find exact matches
exact_matches = gse_cell_lines & gdsc_cell_lines
print(f"\nExact name matches: {len(exact_matches)}")

# Show some examples
if len(exact_matches) > 0:
    print(f"\nExample matches: {list(exact_matches)[:10]}")

In [ ]:
# Analyze name mismatches and create mapping
# Common differences: case sensitivity, hyphens vs underscores, spaces

def normalize_cell_line_name(name):
    """Normalize cell line names for matching"""
    return str(name).upper().replace('-', '').replace('_', '').replace(' ', '')

# Create normalized mappings
gse_normalized = {normalize_cell_line_name(name): name for name in gse_cell_lines}
gdsc_normalized = {normalize_cell_line_name(name): name for name in gdsc_cell_lines}

# Find matches using normalized names
normalized_matches = set(gse_normalized.keys()) & set(gdsc_normalized.keys())
print(f"Matches after normalization: {len(normalized_matches)}")

# Create mapping dictionary: GDSC name -> GSE name
cell_line_mapping = {}
for norm_name in normalized_matches:
    gdsc_name = gdsc_normalized[norm_name]
    gse_name = gse_normalized[norm_name]
    cell_line_mapping[gdsc_name] = gse_name

print(f"\nCreated mapping for {len(cell_line_mapping)} cell lines")
print(f"\nExample mappings:")
for i, (gdsc_name, gse_name) in enumerate(list(cell_line_mapping.items())[:5]):
    print(f"  {gdsc_name} -> {gse_name}")

## 4. Process Drug Response Data

We'll create a matrix with:
- Rows: Cell lines
- Columns: Drugs
- Values: IC50 or AUC (drug sensitivity metrics)

In [ ]:
# Filter GDSC data to matched cell lines only
if len(df_gdsc_raw) > 0:
    df_gdsc_filtered = df_gdsc_raw[df_gdsc_raw['CELL_LINE_NAME'].isin(cell_line_mapping.keys())].copy()
    
    print(f"GDSC entries after filtering to matched cell lines: {len(df_gdsc_filtered):,}")
    print(f"Unique cell lines: {df_gdsc_filtered['CELL_LINE_NAME'].nunique()}")
    print(f"Unique drugs: {df_gdsc_filtered['DRUG_NAME'].nunique()}")
    
    # Map GDSC cell line names to GSE names
    df_gdsc_filtered['GSE_CELL_LINE'] = df_gdsc_filtered['CELL_LINE_NAME'].map(cell_line_mapping)
    
    display(df_gdsc_filtered.head())
else:
    print("No GDSC data to filter. Please download GDSC data first.")

In [ ]:
# Create drug response matrix (cell lines × drugs)
if len(df_gdsc_filtered) > 0:
    # Use LN_IC50 as the primary metric (natural log of IC50)
    # Lower values = more sensitive to drug
    
    df_drug_response = df_gdsc_filtered.pivot_table(
        index='GSE_CELL_LINE',
        columns='DRUG_NAME',
        values='LN_IC50',
        aggfunc='mean'  # Average if multiple measurements
    )
    
    print(f"Drug response matrix shape: {df_drug_response.shape}")
    print(f"Cell lines: {df_drug_response.shape[0]}")
    print(f"Drugs: {df_drug_response.shape[1]}")
    print(f"\nMissing data: {df_drug_response.isna().sum().sum() / df_drug_response.size * 100:.1f}%")
    
    display(df_drug_response.iloc[:5, :5])
else:
    print("Cannot create drug response matrix without GDSC data.")

In [ ]:
# Filter to high-quality drugs
# Criteria: tested in >100 cell lines, <50% missing data

if len(df_drug_response) > 0:
    # Count non-missing values per drug
    drug_coverage = df_drug_response.notna().sum()
    
    # Filter drugs
    min_cell_lines = 100
    high_quality_drugs = drug_coverage[drug_coverage >= min_cell_lines].index
    
    df_drug_response_filtered = df_drug_response[high_quality_drugs]
    
    print(f"High-quality drugs (tested in ≥{min_cell_lines} cell lines): {len(high_quality_drugs)}")
    print(f"\nFiltered drug response shape: {df_drug_response_filtered.shape}")
    print(f"\nTop 10 drugs by coverage:")
    print(drug_coverage.sort_values(ascending=False).head(10))
else:
    print("Skipping drug filtering.")

## 5. Combine Methylation and Drug Response Data

In [ ]:
# For ML purposes, we'll use the top 10,000 most variable CpG sites
# (This was already computed in the EDA notebook)

# Check if pre-computed file exists
top_cpgs_file = Path('../data/processed/GSE68379_top10k_variable_CpGs.csv')

if top_cpgs_file.exists():
    print("Loading pre-computed top 10K variable CpG sites...")
    df_top_cpgs = pd.read_csv(top_cpgs_file, index_col=0)
    top_cpg_names = df_top_cpgs.index.tolist()
    print(f"Loaded {len(top_cpg_names)} CpG sites")
else:
    print("Computing top 10K variable CpG sites...")
    # Calculate variance for each CpG site
    cpg_variance = df_meth.var(axis=1)
    top_cpg_names = cpg_variance.nlargest(10000).index.tolist()
    print(f"Selected top {len(top_cpg_names)} variable CpG sites")

# Extract methylation data for these CpG sites
df_meth_filtered = df_meth.loc[top_cpg_names, :].T  # Transpose: rows=cell lines, cols=CpGs
print(f"\nFiltered methylation data shape: {df_meth_filtered.shape}")
print(f"Rows (cell lines): {df_meth_filtered.shape[0]}")
print(f"Columns (CpG sites): {df_meth_filtered.shape[1]}")

In [ ]:
# Combine methylation and drug response data
# Only keep cell lines that have BOTH methylation AND drug response data

if len(df_drug_response_filtered) > 0:
    # Find common cell lines
    common_cell_lines = df_meth_filtered.index.intersection(df_drug_response_filtered.index)
    
    print(f"Cell lines with both methylation and drug response: {len(common_cell_lines)}")
    
    # Combine data
    df_combined = pd.concat([
        df_meth_filtered.loc[common_cell_lines],
        df_drug_response_filtered.loc[common_cell_lines]
    ], axis=1)
    
    print(f"\nCombined dataset shape: {df_combined.shape}")
    print(f"Rows (cell lines): {df_combined.shape[0]}")
    print(f"Columns (features + targets):")
    print(f"  - Methylation features (CpG sites): {len(top_cpg_names)}")
    print(f"  - Drug response targets: {len(high_quality_drugs)}")
    print(f"  - Total columns: {df_combined.shape[1]}")
    
    display(df_combined.iloc[:5, :5])
else:
    print("Cannot combine data without drug response information.")
    # Create methylation-only dataset
    df_combined = df_meth_filtered
    print(f"\nCreating methylation-only dataset: {df_combined.shape}")

## 6. Add Sample Metadata Columns

In [ ]:
# Add metadata columns (primary site, histology, etc.)
# These can be used as additional features or for stratification

# Match metadata to combined dataset
df_meta_matched = df_meta[df_meta['cell line'].isin(df_combined.index)].copy()
df_meta_matched = df_meta_matched.set_index('cell line')

# Reorder to match df_combined
df_meta_matched = df_meta_matched.loc[df_combined.index]

print(f"Matched metadata for {len(df_meta_matched)} cell lines")
print(f"\nMetadata columns to add:")
for col in df_meta_matched.columns:
    print(f"  - {col}")

display(df_meta_matched.head())

In [ ]:
# Create final dataset with metadata as first columns
# Structure: [metadata columns] [methylation features] [drug response targets]

metadata_cols = ['primary site', 'primary histology', 'cosmic_id']
metadata_subset = df_meta_matched[metadata_cols]

# Combine: metadata + methylation + drug response
df_final = pd.concat([metadata_subset, df_combined], axis=1)

print(f"Final ML-ready dataset shape: {df_final.shape}")
print(f"\nColumn structure:")
print(f"  - Metadata columns: {len(metadata_cols)}")
print(f"  - Methylation features: {len(top_cpg_names)}")
if len(df_drug_response_filtered) > 0:
    print(f"  - Drug response targets: {len(high_quality_drugs)}")
print(f"  - Total columns: {df_final.shape[1]}")

print(f"\nData types:")
print(df_final.dtypes.value_counts())

display(df_final.iloc[:5, :8])

## 7. Data Quality Checks

In [ ]:
# Check for missing values
print("Missing value analysis:\n")

# Metadata
metadata_missing = df_final[metadata_cols].isna().sum()
print("Metadata columns:")
print(metadata_missing)

# Methylation features
meth_cols = [col for col in df_final.columns if col.startswith('cg')]
meth_missing_pct = df_final[meth_cols].isna().sum().sum() / (len(meth_cols) * len(df_final)) * 100
print(f"\nMethylation features: {meth_missing_pct:.2f}% missing")

# Drug response targets
if len(df_drug_response_filtered) > 0:
    drug_cols = [col for col in df_final.columns if col not in metadata_cols and not col.startswith('cg')]
    drug_missing_pct = df_final[drug_cols].isna().sum() / len(df_final) * 100
    print(f"\nDrug response targets (% missing per drug):")
    print(drug_missing_pct.describe())
    print(f"\nDrugs with <10% missing: {(drug_missing_pct < 10).sum()}")

In [ ]:
# Visualize data completeness
if len(df_drug_response_filtered) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Drug coverage histogram
    ax = axes[0]
    drug_coverage_pct = (1 - drug_missing_pct / 100) * 100
    ax.hist(drug_coverage_pct, bins=30, edgecolor='black')
    ax.set_xlabel('Data Completeness (%)')
    ax.set_ylabel('Number of Drugs')
    ax.set_title('Drug Response Data Completeness')
    ax.axvline(90, color='red', linestyle='--', label='90% threshold')
    ax.legend()
    
    # Sample completeness
    ax = axes[1]
    sample_completeness = (1 - df_final[drug_cols].isna().sum(axis=1) / len(drug_cols)) * 100
    ax.hist(sample_completeness, bins=30, edgecolor='black')
    ax.set_xlabel('Data Completeness (%)')
    ax.set_ylabel('Number of Cell Lines')
    ax.set_title('Cell Line Drug Response Completeness')
    
    plt.tight_layout()
    plt.savefig('../data/figures/ML_dataset_completeness.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nCell line completeness summary:")
    print(sample_completeness.describe())

## 8. Save ML-Ready Dataset

In [ ]:
# Save as compressed CSV
output_file = Path('../data/processed/ML_dataset_methylation_drug_response.csv.gz')

print(f"Saving dataset to {output_file}...")
df_final.to_csv(output_file, compression='gzip')

# Check file size
file_size_mb = output_file.stat().st_size / (1024 * 1024)
print(f"File saved: {file_size_mb:.2f} MB")

# Also save metadata separately
metadata_file = Path('../data/processed/ML_dataset_sample_info.csv')
df_meta_matched.to_csv(metadata_file)
print(f"Sample metadata saved: {metadata_file}")

In [ ]:
# Create column name mapping
# This helps identify which columns are features vs targets

column_info = pd.DataFrame({
    'column_name': df_final.columns,
    'column_type': ['metadata'] * len(metadata_cols) + 
                   ['methylation_feature'] * len(meth_cols) +
                   ['drug_response_target'] * (len(df_final.columns) - len(metadata_cols) - len(meth_cols)),
    'data_type': df_final.dtypes.values,
    'missing_count': df_final.isna().sum().values,
    'missing_pct': (df_final.isna().sum() / len(df_final) * 100).values
})

column_info_file = Path('../data/processed/ML_dataset_column_info.csv')
column_info.to_csv(column_info_file, index=False)
print(f"Column info saved: {column_info_file}")

display(column_info.head(10))

## 9. Create Summary Statistics

In [ ]:
# Generate summary statistics for the dataset

summary_stats = {
    'total_samples': len(df_final),
    'total_features': len(meth_cols),
    'total_targets': len(drug_cols) if len(df_drug_response_filtered) > 0 else 0,
    'total_columns': df_final.shape[1],
    'file_size_mb': file_size_mb,
    'methylation_missing_pct': meth_missing_pct,
    'primary_sites': df_final['primary site'].nunique(),
    'histologies': df_final['primary histology'].nunique()
}

print("Dataset Summary:")
print("=" * 50)
for key, value in summary_stats.items():
    print(f"{key:30s}: {value}")

# Save summary
summary_file = Path('../data/processed/ML_dataset_summary.txt')
with open(summary_file, 'w') as f:
    f.write("ML-Ready Dataset Summary\n")
    f.write("=" * 50 + "\n\n")
    for key, value in summary_stats.items():
        f.write(f"{key:30s}: {value}\n")

print(f"\nSummary saved: {summary_file}")

## 10. Quick Validation Test

Test that the dataset can be loaded and used for ML

In [ ]:
# Test loading the saved dataset
print("Testing dataset load...")
df_test = pd.read_csv(output_file, compression='gzip', index_col=0)
print(f"Loaded successfully: {df_test.shape}")

# Verify it matches original
assert df_test.shape == df_final.shape, "Shape mismatch!"
print("✓ Shape verification passed")

# Show example of how to use it
print("\nExample usage:")
print("")
print("# Load dataset")
print("df = pd.read_csv('ML_dataset_methylation_drug_response.csv.gz', index_col=0)")
print("")
print("# Separate features and targets")
print("X = df[[col for col in df.columns if col.startswith('cg')]]  # Methylation features")
print("y = df['DRUG_NAME']  # Target drug (choose from available drugs)")
print("")
print("# Train model")
print("from sklearn.ensemble import RandomForestRegressor")
print("model = RandomForestRegressor()")
print("model.fit(X, y.dropna())  # Remove missing values")

In [ ]:
print("="*60)
print("DATASET PREPARATION COMPLETE")
print("="*60)
print(f"\nFiles created:")
print(f"  1. {output_file} ({file_size_mb:.1f} MB)")
print(f"  2. {metadata_file}")
print(f"  3. {column_info_file}")
print(f"  4. {summary_file}")
print(f"\nReady for ML modeling!")
print(f"\nNext steps for your groupmate:")
print(f"  - Load the dataset using pandas")
print(f"  - Select features (CpG methylation sites)")
print(f"  - Select target (specific drug)")
print(f"  - Train XGBoost/Random Forest/etc.")
print(f"  - Evaluate with cross-validation")